# Mixing matrices

By default the Summer compartmental model assumes that everybody comes into contact with everybody else about the same amount (homogeneous mixing). This isn't always true. It may be the case, for example, that children tend to interact more with other children, and less with the elderly. This difference in social mixing can be expressed and modelled using a "mixing matrix". A NxN matrix which defines how each N strata of a stratification interact and may infect each other.


For example, in a 'child'/'adult' stratification, we could have the following mixing matrix:

| | child | adult |
| --- | --- | --- |
| child | 0.5 | 0.5 |
| adult | 0.5 | 0.5 |


In this mixing matrix, the columns are the infectors and the rows the infected. So the above matrix has following values:

| | child | adult |
| --- | --- | --- |
| child | child -> child | adult -> child |
| adult | child -> adult | adult -> adult |


SEE tests/test_force_of_infection.py

## Prem et al. mixing matrices based on age and location

You can obtain estimated mixing matrices from [Projecting social contact matrices in 152 countries using contact surveys and demographic data](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005697#sec020) by Prem et al. in PLOS Computational Biology in 2017.

This paper is accompanied by age and location specific mixing matrices for 152 countries. You can download the matrices as Excel spreadsheets [here](https://doi.org/10.1371/journal.pcbi.1005697.s002). The paper provides mixing matrices for 5 location types:

- home
- school
- work
- other_locations
- all_locations

The rows and columns indices of each matrix represent a 5 year age bracket from 0-80, giving us a 16x16 matrix. 

A more recent version of these social mixing matrices can be obtained from [Kiesha Prem's GitHub](https://github.com/kieshaprem/synthetic-contact-matrices)